# Усреднение данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import traceback

Названия столбцов и заголовков

In [2]:
timestamp = 'timestamp'
ch4_name = 'CH4'
ch4Ref_name = 'CH4Ref'
ch4LR_name = 'CH4LR'
voltage_name = 'V'
adc_name = 'ADC'
T_name = 'T'
rh_name = 'rH'
ah_name = 'aH'
P_name = 'P'
fanspeed_name = 'fanSpeed'
res_header = [timestamp, adc_name, voltage_name, T_name, rh_name, ah_name, P_name, fanspeed_name]

# Настройки усреднения

Порядок усреднения величин

In [ ]:
demands_ref = {timestamp: 'first', ch4Ref_name: 'mean'}
demands_data = {timestamp: 'first', adc_name: 'mean', T_name: 'mean', rh_name: 'mean', ah_name: 'mean', P_name: 'mean', fanspeed_name: 'mean'}
round_order = {adc_name : 0, T_name : 1, rh_name : 1, ah_name : 5, P_name : 1}
# если длительность измерения превышает 60 секунд (т.е. количество точек при измерении 1 раз в секунду), округлять данные каждые 30 секунд
max_time_for_one_point = 60
averaging_period = 30

Задать id серий (списком или диапазоном) и названия папок

In [7]:
seriesIds = range(28,29)
folder_name = "/averaged/" # имя папки с результатами (усредненными данными)
ref_folder = "processed_auto" # имя папки с данными для усреднения
MS_DATA_path = "../../MS_DATA" # путь до папки с сериями

Не менять

In [ ]:
voltage_step = 2.5 / 2**16 # шаг по напряжению (разница между двумя соседними отсчетами)

# Функции

Функции для построения графиков

In [4]:
def plotRefAndMeasure(x1, y1, x2, y2, title):
	figure, axis = plt.subplots(2, 1, sharex=True, figsize=(16,10), dpi = 300)
	axis[0].scatter(x1, y1)
	axis[0].set_title("Газоанализатор ABB (эталонный прибор)")
	axis[0].set_ylabel('Содержание метана, ppm')
	axis[1].scatter(x2, y2)
	axis[1].set_title("Калибруемый датчик")
	axis[1].set_ylabel('Напряжение, В')
	figure.suptitle(title)
	return figure

def plotMeasure(x1, y1, title):
	figure, axis = plt.subplots(figsize=(16,10), dpi = 300)
	axis.scatter(x1, y1)
	plt.xlabel('Время, с')
	plt.ylabel('Напряжение, В')
	plt.title(title)
	return figure

Функция для конкатинации

In [5]:
def concatRefDataIntoFrame(dirPath):
	all_files = []
	if os.path.isdir(dirPath):
		all_files = glob.glob(dirPath + "/refData*.csv")
	else:
		all_files = [dirPath]
	li = []
	for filename in all_files:
		df = pd.read_csv(filename, delimiter=',', header=None)
		li.append(df)
	frame = pd.concat(li, ignore_index=True)
	return frame

# Усреднение и сохранение результатов

In [14]:
for seriesId in seriesIds:
	try:
		seriesDataTemplate = f"{MS_DATA_path}/series{seriesId}_*"
		seriesDataPath = glob.glob(seriesDataTemplate)[0]
		seriesRefDataTemplate = f"{MS_DATA_path}/ReferenceData/referenceData_series{seriesId}_*"
		dirName = seriesDataPath + folder_name
		measurePathes = glob.glob(seriesDataPath + "/measure*.csv")
		measurementsCount = len(measurePathes)
		#Создание папки внутри серии для усредненных данных
		if not os.path.exists(dirName):
			os.makedirs(dirName)
			print("Directory " , dirName ,  " Created ")

		for measureId in range(1, measurementsCount + 1):
			# по дефолту округляем до одной точки
			number = 1
			try:
				measureTemplate = seriesDataPath + f"/measure{measureId}_*.csv"
				measureDataPath = glob.glob(measureTemplate)[0]
				measureRefDataTemplate = seriesDataPath + f"/{ref_folder}/refData_measure{measureId}_*"
				measureRefDataPathes = glob.glob(measureRefDataTemplate)
				name = os.path.basename(measureDataPath)
				measure_name = os.path.splitext(name)[0]
				image_path = measure_name +'_row.png'

				df_measure = pd.read_csv(measureDataPath)
				df_measure[timestamp] = pd.to_datetime(df_measure[timestamp])
				if measureRefDataPathes:
					measureRefDataPath = glob.glob(measureRefDataTemplate)[0]
					df_ref = pd.read_csv(measureRefDataPath, header=None)
					df_ref.set_axis([timestamp, ch4Ref_name], axis='columns', inplace=True)
					df_ref[timestamp] = pd.to_datetime(df_ref[timestamp])
					figure_before = plotRefAndMeasure(df_ref.timestamp, df_ref.CH4Ref, df_measure.timestamp, df_measure.V, f"Данные измерения №{measureId} из серии {seriesId} без усреднения")
					# если длительность измерения превышает 60 секунд (т.е. количество точек при измерении 1 раз в секунду), округлять данные каждые 30 секунд
					if len(df_ref) / max_time_for_one_point > 1 :
						number = int(len(df_ref) / averaging_period)
					# количество точек в одном сэмпле усреднения
					samples_number = len(df_ref.index) // number
					res_ref = df_ref.groupby(df_ref.index // samples_number).agg(demands_ref)
					res_ref = res_ref.round({ch4Ref_name : 3})
					# если количество точек не делится нацело, то остается 1 или больше точек, которые не округляются, их нужно удалить
					if len(res_ref) > number:
						res_ref.drop(res_ref.tail(len(res_ref) - number).index,inplace=True)
					res_ref.to_csv(dirName + "/" + f"refData_{name}", header=None, index=None)
				else:
					# если нет референсных данных
					figure_before = plotMeasure(df_measure.timestamp, df_measure.V, f"Данные измерения №{measureId} из серии {seriesId} без усреднения")
					if len(df_measure) / max_time_for_one_point > 1 :
						number = int(len(df_measure) / averaging_period)
					samples_number = len(df_measure.index) // number
				figure_before.savefig(dirName + "/" + measure_name +'_row.png')
				# отключает отображение графиков в скрипте, можно закомментировать 2 следующие строки
				plt.close()
				plt.ioff()

				res_measure = df_measure.groupby(df_measure.index // samples_number).agg(demands_data)
				res_measure = res_measure.round(round_order)
				# расчет напряжения по усредненным значениям отсчетов АЦП, чтобы не получались значения в промежутках между возможными значениями напряжения
				# шаг по напряжению: 0.00003 В
				res_measure[voltage_name] = round(res_measure[adc_name] * voltage_step, 5)
				res_measure = res_measure.astype({adc_name :'int'})
				res_measure = res_measure[res_header]
				# если длительность измерения превышает 60 секунд (т.е. количество точек при измерении 1 раз в секунду), округлять данные каждые 30 секунд
				if len(res_measure) > number:
					res_measure.drop(res_measure.tail(len(res_measure) - number).index,inplace=True)
				res_measure.to_csv(dirName + "/" + name, index= None)
				if measureRefDataPathes:
					figure = plotRefAndMeasure(res_ref[timestamp], res_ref[ch4Ref_name], res_measure[timestamp], res_measure[voltage_name], f"Данные измерения №{measureId} из серии {seriesId} после усреднения")
				else:
					figure = plotMeasure(res_measure[timestamp], res_measure[voltage_name], f"Данные измерения №{measureId} из серии {seriesId} после усреднения")
				figure.savefig(dirName + "/" + measure_name +'_averaged.png')
				# отключает отображение графиков в скрипте, можно закомментировать 2 следующие строки
				plt.close()
				plt.ioff()

			except Exception as e:
				print(measureId, e)
				continue
				
		#Конкатинация референсных данных для отдельных измерений в один файл для всей серии
		if measureRefDataPathes:
			endRefPath = dirName + "referenceData_{}.csv".format(os.path.splitext(os.path.basename(seriesDataPath))[0])
			seriesRefDf = concatRefDataIntoFrame(dirName)
			seriesRefDf.set_axis([timestamp, ch4Ref_name], axis='columns', inplace=True)
			seriesRefDf.sort_values(by=[timestamp], inplace=True)
			seriesRefDf = seriesRefDf.round({ch4Ref_name : 3})
			seriesRefDf.to_csv(endRefPath, header=None, index=None)


	except Exception as e:
		print(seriesId, traceback.format_exc())
		continue
	finally:
		df_ref = None
		df_measure = None